In [0]:

import zlib
import json

def generate_source():
    
    from aadatapipelinecore.core.fs.device.s3 import S3Bucket
    from aadatapipelinecore.core.utils.dict_ import ADDict
    from aadatapipelinecore.core.utils import avro_
    import json
    res_list = list()
    conf = ADDict({"bucket_name": "prod_appannie_ios"})
    bucket = S3Bucket(conf)
    raw_path = "featured_data_ios14/2021-07-05/AU/games.json.gz"
   
    bytes = bucket.get(raw_path)
    res = json.loads(zlib.decompress(bytes,16+zlib.MAX_WBITS))
    return res
       
#     bytes = bucket.get(raw_path)
#     data = json.loads(zlib.decompress(bytes,16+zlib.MAX_WBITS))
#     return data
json.dumps(generate_source(), ensure_ascii=False)

In [0]:
%%sh
# aws s3 ls s3://aardvark-prod-dca-data/tmp/loc_app_detail_more_fields/

aws configure list

# whoami


In [0]:
%%sh
# unified data
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.product-download-channel.v1/fact/granularity_code=daily/
## market_code  apple-store google-play

# aws s3 ls s3://b2c-prod-data-pipeline-unified-featured/unified/featured.ios14-today.v1/fact/granularity=daily/year=2021/

aws s3 ls  s3://prod_appannie_ios/featured_data_ios14/2021-05-01/TR/raw/today/today.json.gz



In [0]:


from  pyspark.storagelevel import StorageLevel

df = spark.read.parquet("s3://aardvark-prod-dca-data/tmp/loc_app_detail")

df.createOrReplaceTempView("loc_app_detail")
df.printSchema()


df_id = spark.sql("select distinct country, app_id, platform, bundle_id from loc_app_detail")
df_name_list = ["name", "installs", "customer_also_bought_apps", "current_release_date", "description", "primary_category_code","icon_url","release_notes","primary_category_id","user_ratings","company_code","price","company_id","related_apps","size","version","screenshots_json"]

for name in df_name_list:
    sql_str = "select * from loc_app_detail where field_name='{}'".format(name)
    df_tmp_field = spark.sql(sql_str)
    df_id = df_id.join(df_tmp_field, on=["country", "app_id", "platform", "bundle_id"], how='left').drop("field_name").withColumnRenamed("value", name)

df_id.show(10)


In [0]:


import sys
sys.path.append("/mnt/application_name=bdp_fredric_test/application_code/version=latest/code/bdp_resources/python/")
import vector_add

c = vector_add.CT(range(1, 2000000))
res = c.vector_add()

res







In [0]:
%%sh



In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.26.136 -U app_bdp_usage_qa -d aa -p 6432 << EOF 
--select * from plproxy.execute_select(\$proxy\$select date, app_id, city_id, est_average_active_users ,est_average_bytes_per_user
--               from mu.city_app_w where city_id='31248510' and app_id='20600000304614'  and country_code='ae' and date='2020-02-29'
--                \$proxy\$) t (date date, 
--               app_id bigint, city_id bigint, est_average_active_users real , est_average_bytes_per_user real) limit 3;
-- SELECT domain_id AS domain_id, name AS domain_name FROM aa_domain_metadata WHERE is_disabled IN ('f') and sensitive_status IN (0);
-- \d aa_domain_metadata
SELECT  domain_id , update_time  FROM aa_domain_metadata WHERE is_disabled IN ('f') and sensitive_status IN (0) and domain_id in ( '70010000208830' , '70010000210592', '70010000002208' , '70010000021237', '70010000000321', '70010000211341', '70010000001238');
SELECT  *  FROM aa_domain_metadata WHERE is_disabled IN ('f') and sensitive_status IN (0) limit 3

EOF


In [0]:

from pyspark.sql import Window
from pyspark.sql.functions import *


windSpec = (Window.partitionBy(lit(0))
                     .orderBy(monotonically_increasing_id()))
                     
# snowflake production
sfOptions = {
  "sfURL" : "appannie_aa_int_prod.us-east-1.snowflakecomputing.com",
  "sfUser" : "app_bdp_data_validation_qa",
  "sfPassword" : "0HN#s@Wa5$1R8jVj",
  "sfDatabase" : "AA_INTELLIGENCE_PRODUCTION",
  "sfSchema" : "ADL_STORE_PAID",
  "sfWarehouse" : "wh_dod_read7"
}


SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"
df = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
  .options(**sfOptions) \
  .option("query",  "select * from fact_store_product_download_channel_t_q_v1_cluster_by_date") \
  .load()

df1 = df.filter((col("COUNTRY_CODE")!='WW') & (col('device_code') == 'ios-phone')).orderBy(df.EST_DOWNLOAD.desc()).limit(100)
df2 = df1.withColumn("row_num", row_number().over(windSpec))


df2.select("date_key").show(1, False)

ls = []
# 10月
for i in range(1, 32):
    ls.append(20201000+i)

# 11月 
for i in range(1, 31):
    ls.append(20201100+i)

# 12月
for i in range(1, 32):
    ls.append(20201200+i)


days = [{'date_key':20201231, 'days': ls }]
df_date = spark.createDataFrame(days)

'''
df_date 表的结构, 用于展开后构造和 daily表join的DataFrame

date_key          days
20210227          [20210221, 20210222, .... 20210227]
'''

# 验证前3条quarterly数据是正确的
for i in range(0, 1):
    # 取第i个10条
    df_inner = df2.filter((col("row_num")>= i*3) & (col('row_num') < (i+1)*3))
    
    inner_res_df = (df_inner.join(df_date, ["date_key"], "left").withColumnRenamed("date_key", "date_key_weekly").withColumn("date_key", explode("days")).withColumnRenamed("est_organic_download", "est_organic_download_weekly").
            withColumnRenamed("est_download", "est_download_weekly").withColumnRenamed("est_paid_download", "est_paid_download_weekly")
        )
    
    df_day = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
    .options(**sfOptions) \
    .option("query",  "select * from FACT_STORE_PRODUCT_DOWNLOAD_CHANNEL_V1_CLUSTER_BY_DATE where date_key>={} and date_key<={}".format(20201001, 20201231)) \
    .load()
    

    df_pre_cal = (inner_res_df.join(df_day, ["date_key", "product_key", "device_key"], how='left').
            groupBy('product_key', 'date_key_weekly', "device_key").agg(sum("est_download").alias("est_download_sum"), sum('est_organic_download').alias("est_organic_download_sum"),
            sum('est_paid_download').alias("est_paid_download_sum")) )
    inner_res_df.show(10, False)
    final_res_df = inner_res_df.join(df_pre_cal, ["product_key", "device_key", "date_key_weekly"], how="left").filter("est_organic_download_weekly==est_organic_download_sum and est_paid_download_weekly==est_paid_download_sum and est_download_weekly==est_download_sum")
    print(final_res_df.count())
    


In [0]:

from pyspark.sql.functions import *
from pyspark.storagelevel import StorageLevel

games_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-featured/unified/featured.ios14-games.v1/fact/")
games_df.persist(StorageLevel.MEMORY_AND_DISK)
games_df.filter("featured_us_category_key is null").count()
games_df.select("featured_us_category_key").distinct().show(10000)


In [0]:


from pyspark.sql.functions import *
from pyspark.storagelevel import StorageLevel

games_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-featured/unified/featured.ios14-apps.v1/fact/")

games_df.persist(StorageLevel.MEMORY_AND_DISK)
print(games_df.filter("featured_us_category_key is null").count())
games_df.select("featured_us_category_key").distinct().show(10000)

